# Combine models

In [1]:
import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import csv

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

%load_ext autoreload
%autoreload 2
%matplotlib inline

from helpers import load_data

In [2]:
PREDICTION_PATH = '../data/predictions/'

In [18]:
models = [['mf_als_k20_l01', 4],
         ['recommend_k20_l085_50', 4],
         ['mf_svd_k16', 1],
         ['mf_svd2_k11_r42', 0.5],
         ['mf_svd2_k12_r42', 0.5]]

In [19]:
total_weights = 0
i=0
submissions = {}
for m in models:
    submissions[i] = load_data('{dp}{m}.csv'.format(dp=PREDICTION_PATH, m=m[0]))
    total_weights += m[1]
    i += 1
    
weights = [m[1]/total_weights for m in models]

number of items: 10000, number of users: 1000
number of items: 10000, number of users: 1000
number of items: 10000, number of users: 1000
number of items: 10000, number of users: 1000
number of items: 10000, number of users: 1000


In [20]:
matrix_result = weights[0] * submissions[0].tocsr()

In [21]:
for i in range(1,len(submissions)):
    matrix_result += weights[i] * submissions[i].tocsr()

Create normal averaged submission

In [22]:
sub_filename = 'combination_als4_rec4_svd1_svd21'
(rows, cols, vals) = sp.find(matrix_result)

with open('{dp}{fn}.csv'.format(dp=PREDICTION_PATH, fn=sub_filename), 'w') as csvfile:
    fieldnames = ['Id', 'Prediction']
    writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
    writer.writeheader()
    for (i, u, v) in zip(rows, cols, vals):
        writer.writerow({'Id':'r{r}_c{c}'.format(r=i+1,c=u+1),'Prediction':v})

Create rounded submission:

In [28]:
sub_filename = 'combination_svd_rounded_005'
(rows, cols, vals) = sp.find(matrix_result)

with open('{dp}{fn}.csv'.format(dp=PREDICTION_PATH, fn=sub_filename), 'w') as csvfile:
    fieldnames = ['Id', 'Prediction']
    writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
    writer.writeheader()
    for (i, u, v) in zip(rows, cols, vals):
        intval = round(v)
        if abs(intval - v) < 0.05:
            pred = intval
        else:
            pred = v
        writer.writerow({'Id':'r{r}_c{c}'.format(r=i+1,c=u+1),'Prediction':pred})